# Auto-unseal using AWS KMS

description:

In this tutorial, we'll show an example of how to use Terraform to provision
an instance that can utilize an encryption key from AWS Key Management Services
to unseal Vault.

---

When a Vault server is started, it starts in a [**_sealed_**](https://www.vaultproject.io/docs/concepts/seal) state and it does not know how to decrypt data. Before any operation can be performed on the Vault, it must be unsealed. Unsealing is the process of constructing the master key necessary to decrypt the data encryption key.

![Unseal with Shamir's Secret Sharing](https://content.hashicorp.com/api/assets?product=tutorials&version=main&asset=public%2Fimg%2Fvault%2Fvault-autounseal.png)

This tutorial demonstrates an example of how to use Terraform to provision an
instance that can utilize an encryption key from [AWS Key Management Services
(KMS)](https://aws.amazon.com/kms/) to unseal Vault.

## Challenge

Vault unseal operation requires a quorum of existing unseal keys split by
Shamir's Secret sharing algorithm. This is done so that the "_keys to the
kingdom_" won't fall into one person's hand. However, this process is manual
and can become painful when you have many Vault clusters as there are now
many different key holders with many different keys.

## Solution

Vault supports opt-in automatic unsealing via cloud technologies: AliCloud KMS,
AWS KMS, Azure Key Vault, Google Cloud KMS, and OCI KMS. This feature enables
operators to delegate the unsealing process to trusted cloud providers to ease
operations in the event of partial failure and to aid in the creation of new or
ephemeral clusters.

![Unseal with AWS KMS](https://content.hashicorp.com/api/assets?product=tutorials&version=main&asset=public%2Fimg%2Fvault%2Fvault-autounseal-2.png)

This tutorial demonstrates Vault Auto Unseal using AWS KMS. 

## Prerequisites

This tutorial assumes the following:

- AWS account for provisioning cloud resources 
- [Terraform installed](https://www.terraform.io/downloads) and basic understanding of
  its usage
- docker and docker-compose

> **NOTE**: [Seal
migration](https://www.vaultproject.io/docs/concepts/seal#seal-migration) from
Auto Unseal to Auto Unseal of the same type is supported since Vault 1.6.0.
However, there is a current limitation that prevents migrating from AWS KMS to
AWS KMS; all other seal migrations of the same type are supported. Seal
migration from one Auto Unseal type (AWS KMS) to another Auto Unseal type
(HSM, Azure KMS, etc.) is also supported on older versions as well.

## Step 1: Provision the cloud resources

### AWS Credentials

Set your AWS Credentials. I got one from Instruqt terminal with this command.

```bash
env | grep -iE "^aws.*access" | xargs -I{} echo export {}
```

In [ ]:
env | grep -iE "^aws.*access" | xargs -I{} echo export {}

In [ ]:
unset AWS_ACCESS_KEY_ID AWS_SECRET_ACCESS_KEY
export AWS_DEFAULT_REGION=us-west-2
export AWS_REGION=$AWS_DEFAULT_REGION
export AWS_ACCESS_KEY_ID=AKIAROSAM
export AWS_SECRET_ACCESS_KEY=vqzZ5zhwqWTZy04zD

printf "#==> Creds: \n$AWS_REGION \n$AWS_ACCESS_KEY_ID\n$AWS_SECRET_ACCESS_KEY\n"

aws configure set region us-west-2 --profile default

> **Tip:** The above example uses IAM user authentication. You can use any authentication method described in the [AWS provider documentation](https://registry.terraform.io/providers/hashicorp/aws/latest/docs#environment-variables).

Create default VPC if needed.

In [ ]:
aws ec2 create-default-vpc | jq -c || true

In [ ]:
printf "\n#==> Show VPC ids\n"
aws ec2 describe-vpcs | jq -r '.[] | .[] | .VpcId'

### terraform configuration

Build terraform configuration to provision AWS kms key.

In [ ]:
pushd /tmp/ssh/
cat > main.tf <<"EOF"
provider "aws" {
}
resource "random_pet" "env" {
  length    = 2
  separator = "_"
}
resource "aws_kms_key" "vault" {
  description             = "Vault unseal key"
  deletion_window_in_days = 10

  tags = {
    Name = "vault-kms-unseal-${random_pet.env.id}"
  }
}
output "vault_key_arn" {
  value = aws_kms_key.vault.arn
}
output "vault_key_id" {
  description = "The globally unique identifier for the key"
  value       = aws_kms_key.vault.id
}
EOF
# terraform init
popd

1. Perform a `terraform init` to pull down the necessary provider resources.

In [ ]:
pushd /tmp/ssh/
terraform init
popd

Then `terraform plan` to verify your changes and the resources that will be
created.

In [ ]:
terraform -chdir=/tmp/ssh plan

### terraform apply

1. Run `terraform apply` and review the planned actions. Your terminal output
   should indicate the plan is running and what resources will be created.

In [ ]:
terraform -chdir=/tmp/ssh/ apply -auto-approve

When the `apply` command completes, the Terraform output will display the AWS KMS key id and arn.
   
```plaintext
Outputs:
vault_key_arn = "arn:aws:kms:us-west-2:099993004052:key/303daec2-43e4-414e-8238-d30a20595eca"
vault_key_id = "303daec2-43e4-414e-8238-d30a20595eca"
```

### docker compose

Create a `docker-compose.yaml` file.

In [ ]:
export VAULT_PORT=8200 #8200 is default
export VAULT_ADDR=http://127.0.0.1:${VAULT_PORT:=8200}
export VAULT_TOKEN_PRIMARY=myroot #used to keep track of root token
printf "VAULT TOKEN for root is: $VAULT_TOKEN_PRIMARY \n"
export VAULT_TOKEN=${VAULT_TOKEN_PRIMARY:=root}
printf "\n#==> Please Run: export VAULT_TOKEN=${VAULT_TOKEN_PRIMARY}\n"
printf "VAULT_ADDR=${VAULT_ADDR}\n"

# Common
export VAULT_VER=1.10.4 # 1.8+ enterprise requires license file; 1.7.5 has 8 hour eval
export VAULT_LICENSE=$(cat ../../license/vault.hclic)

In [ ]:
## Create docker-compose file.
WORK_DIR=/tmp/ssh
mkdir -p ${WORK_DIR}/{data,logs,config}
tee /tmp/ssh/docker-compose.yaml << EOF
version: '3.8'
services:
  vault_server:
    #image: hashicorp/vault-enterprise:${VAULT_VER}_ent # Vault Enterprise
    image: hashicorp/vault:${VAULT_VER}                # Vault OSS
    container_name: vault_server
    hostname: vault_server
    restart: always
    volumes:
#      - ./${WORK_DIR}/data:/vault/data # uncomment to persist data
      - ${WORK_DIR}/logs:/vault/logs
      - ${WORK_DIR}/config:/vault/config:ro # uncomment for local config
    ports:
      - "$VAULT_PORT:8200/tcp"
    environment:
      VAULT_DEV_ROOT_TOKEN_ID: ${VAULT_TOKEN:-root}
      VAULT_DEV_LISTEN_ADDRESS: "0.0.0.0:8200"
      VAULT_ADDR: http://127.0.0.1:8200
      VAULT_LICENSE: ${VAULT_LICENSE}
      AWS_ACCESS_KEY_ID: ${AWS_ACCESS_KEY_ID}
      AWS_SECRET_ACCESS_KEY: ${AWS_SECRET_ACCESS_KEY}
      AWS_REGION: ${AWS_REGION}
    cap_add:
      - IPC_LOCK
    #entrypoint: "vault server -dev" # dev mode
    entrypoint: "vault server -config=/vault/config" # non-dev with local config
    networks:
      vault_net:
        ipv4_address: 172.16.238.10
networks:
  vault_net:
    driver: bridge
    ipam:
      driver: default  
      config:
      - subnet: 172.16.238.0/24
EOF

### vault config file

Create the Vault configuration file.

In [ ]:
tee /tmp/ssh/config/vault.hcl <<EOF
storage "file" {
  path = "/vault/data"
}

listener "tcp" {
  address     = "0.0.0.0:8200"
  tls_disable = "true"
}

seal "awskms" {
  #disabled   = "true"
  region     = "${AWS_REGION}"
  kms_key_id = "$(terraform -chdir=/tmp/ssh/ output -raw vault_key_id)"
}

ui=true
log_level = "DEBUG"
EOF

**NOTE**:
- I specified the AWS creds in the container environment instead on in the `seal` stanza.
    - See `docker-compose.yml`
- The Vault configuration file defines the `awskms` stanza
    - `kms_key_id` - sets the AWS KMS key ID to use for encryption and decryption. This value is pulled from the `terraform output`.

Start up Vault Enterprise as a container.

In [ ]:
pushd /tmp/ssh/
docker-compose up --force-recreate --build -d \
  vault_server 
popd

Verify the docker container `vault_server` is running.

In [ ]:
docker ps | grep -E "vault_server"

In [ ]:
docker exec -it vault_server vault status || true

**NOTE** Initialized (`false`) and Sealed (`true`) status 

## Step 2: Test the auto-unseal feature

Verify that Vault has been installed, run `vault status` command.

In [ ]:
vault status || true

**NOTE**:
- `Recovery Seal Type`is `awskms`
- `Initialized` is `false`
- `Sealed` is true

Sample Output
```text
Key                      Value
---                      -----
Recovery Seal Type       awskms
Initialized              false
Sealed                   true
Total Recovery Shares    0
Threshold                0
Unseal Progress          0/0
Unseal Nonce             n/a
Version                  1.10.4
Storage Type             file
HA Enabled               false
```

Run the `vault operator init` command to initialize the Vault server.

In [ ]:
vault operator init -format=json | tee /tmp/ssh/vault_init.txt

<details><summary><b>Example output:</b></summary>

```plaintext
Recovery Key 1: rbzYWWvegCxiZWAGPwDJJAmg6GOnBBnW9QrgKTLLn/eE
Recovery Key 2: +ipeyG4OmSCzl0zbriFuTYGhB1AP7fGkmzLkd5r6z3OA
Recovery Key 3: jURIBlqENHG2FIWWl/JM8WEJxGPFuBrb4c8Ht1I3a1Ue
Recovery Key 4: lvDSVfwvMcCWUL3kP92ypd3I1Ffaq0JP5Q80HFuBty5v
Recovery Key 5: OImPCrj7H+lnwlyxdbZyNFjjCvjPfx+Xb77r+3HzDes/

Initial Root Token: hvs.GhkPkNSOuvQ8WVcBE6qgtjTC

Success! Vault is initialized

Recovery key initialized with 5 key shares and a key threshold of 3. Please
securely distribute the key shares printed above.
```
</details>

> **NOTE**
> - The initialization generates **Recovery Keys** (instead of **Unseal
 Keys**) when using auto-unseal.
> - Some of the Vault operations still require Shamir keys.
    - For example, to [regenerate a root
 token](/tutorials/vault/policies), each key holder must enter their recovery
 key.
> - Similar to unseal keys, you can specify the number of recovery keys and the threshold using the `-recovery-shares` and `-recovery-threshold` flags.

In [ ]:
for i in {0..2}; do
export KEY_${i}="$(jq -r .recovery_keys_b64[${i}] /tmp/ssh/vault_init.txt)"
done
export ROOT_TOKEN=$(jq -r .root_token /tmp/ssh/vault_init.txt)
printf "%s\n" "$KEY_0" "$KEY_1" "$KEY_2" "$ROOT_TOKEN"

Once you initialized the Vault server, it is ready for operation.

Check the Vault status to verify that it has been initialized and unsealed.

In [ ]:
vault status

Sample Output

```shell
Key                      Value
---                      -----
Recovery Seal Type       shamir
Initialized              true
Sealed                   false
Total Recovery Shares    5
Threshold                3
Version                  1.10.4
Storage Type             file
Cluster Name             vault-cluster-b1c45c4d
Cluster ID               5f26df9c-e0fc-5399-2f7c-6d2613d432c9
HA Enabled               false
```

**NOTE**
- `Recovery Seal Type` is now `shamir`.
  - Previously, it was `awskms`.
- The Vault server is already unsealed (**Sealed** is `false`).

Restart the Vault server to ensure that Vault server gets automatically unsealed:

In [ ]:
docker restart vault_server

Verify the Vault status.

In [ ]:
vault status

**NOTE**: The server is unsealed without you having to enter in any keys.

Log into Vault. Enter the generated initial root token when prompted.

In [ ]:
vault login $ROOT_TOKEN

**Example output:** 

```plaintext   
Key                  Value
---                  -----
token                hvs.O4DWXANNAxpALpLmbYTM5Zfw
token_accessor       bngdvrF09IXZG2a2doDbskXi
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]
```

Review the Vault configuration file (`/vault/config/vault.hcl`).

In [ ]:
docker exec -it vault_server cat /vault/config/vault.hcl

```shell
storage "file" {
  path = "/vault/data"
}
listener "tcp" {
  address     = "0.0.0.0:8200"
  tls_disable = "true"
}
seal "awskms" {
  region = "us-west-2"
  kms_key_id = "b6ef3a34-f8db-4bfd-849a-86690711055e"
}
ui=true
```

Notice the Vault configuration file defines the [`awskms`
stanza](https://www.vaultproject.io/docs/configuration/seal/awskms) which
sets the AWS KMS key ID to use for encryption and decryption.

> Although the listener stanza disables TLS (`tls_disable = "true"`) for this
tutorial, Vault should always be [used with
TLS](https://www.vaultproject.io/docs/configuration/listener/tcp#tls_cert_file)
in production to provide secure communication between clients and the Vault
server. It requires a certificate file and key file on each Vault host.

At this point, you should be able to launch the Vault Enterprise UI by going to the UI

In [ ]:
echo http://$(hostname -I | awk '{print $1}'):8200

Log in with your initial root token.

In [ ]:
echo ROOT_TOKEN=$ROOT_TOKEN

# Seal Migration

## Migrate from Cloud Seal Type to Shamir Seal Type

While not as common as the first case, but also handy for multiple migration paths, this example shows how one might migrate from an existing AWS KMS seal type to the Shamir seal type.

1. Stop **ALL** Vault servers

In [ ]:
docker stop vault_server

2. Update the Vault server's configuration's [`seal` stanza](https://www.vaultproject.io/docs/configuration/seal/index.html) to include a `disable = "true"` option.
    - **NOTE** - `true` is a string and must be quoted in this case. 
1. Updating this can be done in one of two ways:
    1. If you use a single configuration file, you must update it and add the `disable = "true"` option to the `seal` stanza.
    1. If your Vault configuration is modular / consists of multiple configuration files, edit the appropriate file and add `disabled = "true"` to the seal stanza to update it.

In [ ]:
tee /tmp/ssh/config/vault.hcl <<EOF
storage "file" {
  path = "/vault/data"
}

listener "tcp" {
  address     = "0.0.0.0:8200"
  tls_disable = "true"
}

seal "awskms" {
  disabled   = "true"
  region     = "${AWS_REGION}"
  kms_key_id = "$(terraform -chdir=/tmp/ssh/ output -raw vault_key_id)"
}

ui=true
log_level = "DEBUG"
EOF

**NOTE**:
- I specified the AWS creds in the container environment instead on in the `seal` stanza.
- The Vault configuration file defines the `awskms` stanza
    - `kms_key_id` - sets the AWS KMS key ID to use for encryption and decryption.

4. Start a single Vault server

In [ ]:
docker start vault_server

Check vault server status.

In [ ]:
vault status

**NOTE**: The vault server now requires manual unsealing.

5. Initiate an unseal using the command and enter the first recovery key: `vault operator unseal -migrate`

In [ ]:
vault operator unseal -migrate $KEY_0

6. Enter all the remaining unseal keys; these keys must also be entered with the `-migrate` flag ensuring everyone involved in the migration is aware

In [ ]:
vault operator unseal -migrate $KEY_1
vault operator unseal -migrate $KEY_2

7. Observe that the Vault server is functional with e.g. `vault status` and by login / secret retrieval, etc.

In [ ]:
vault status

- `Seal Type` is `shamir` instead of `Recovery Seal Type` `shamir`

In [ ]:
vault login $ROOT_TOKEN

8. Provided that the Vault server appears operational, start and unseal remaining servers in the same way

Once all recovery keys have been entered, the encryption key will be encrypted with the recovery key generated master key, the recovery key and stored Auto Unseal master key will be deleted, and Vault will again become unsealed.

When disabling the seal, it will no longer be used to unseal Vault but the credentials will still be available for any migration that is issued.

## Migrate from Shamir Type to Cloud Seal Type

The most common case for migrate is to update a Shamir seal-based Vault to use a cloud seal with auto-unseal capabilities.

In the following example the process is described for migrating from an existing Shamir seal type to GCP KMS seal type.

1. Stop **ALL** Vault servers
1. Update all of the Vault servers’ configuration to include the appropriate [`seal` stanza](https://www.vaultproject.io/docs/configuration/seal/index.html)
    - see the seal Stanza and documentation for the seal you will use.
1. In this case, we’d consult the [`awskms` Seal documentation](https://www.vaultproject.io/docs/configuration/seal/awskms); updating this can be done in one of two ways:
    1. If you use a single configuration file, you must update it and add the `seal` stanza to the top-level configuration, not under `listener` or `storage`, for example
    1. If your Vault configuration is modular / consists of multiple configuration files, consider adding a new file such as `seal.hcl` with the `seal` stanza contents instead


In [ ]:
tee /tmp/ssh/config/vault.hcl <<EOF
storage "file" {
  path = "/vault/data"
}
listener "tcp" {
  address     = "0.0.0.0:8200"
  tls_disable = "true"
}
seal "awskms" {
  region     = "${AWS_REGION}"
  kms_key_id = "$(terraform -chdir=/tmp/ssh/ output -raw vault_key_id)"
}
ui=true
log_level = "DEBUG"
EOF

**NOTE**:
- I specified the AWS creds in the container environment instead on in the `seal` stanza.
- The Vault configuration file defines the `awskms` stanza
    - `kms_key_id` - sets the AWS KMS key ID to use for encryption and decryption.

4. Start a single Vault server

In [ ]:
docker restart vault_server

5. Unseal the server using the following command and enter the first recovery key: `vault operator unseal -migrate`

In [ ]:
vault operator unseal -migrate $KEY_0

6. Enter all the remaining unseal keys; these keys must also be entered with the `-migrate` flag ensuring everyone involved in the migration is aware

In [ ]:
vault operator unseal -migrate $KEY_1
vault operator unseal -migrate $KEY_2

7. Observe that the Vault server is functional with e.g. `vault status` and by login / secret retrieval, etc.

In [ ]:
vault status

- `Recovery Seal Type` instead of `Seal Type`.

In [ ]:
vault login $ROOT_TOKEN

8. Provided that the Vault server appears operational, start and unseal remaining servers in the same way

Once all unseal keys are provided, and the `migrate` flag is present, the current Shamir shares will become the Recovery Keys for the seal and a new master key will be generated, the encryption key will be encrypted with the new master key, and the master key will be encrypted with the seal and stored in the storage backend.

Subsequent restarting of Vault servers going forward should then result in them automatically unsealing.

# Step 3: Clean up

Stop containers

In [ ]:
pushd /tmp/ssh/
docker-compose down
popd

Delete the AWS cloud resources provisioned by the Terraform files.

In [ ]:
terraform -chdir=/tmp/ssh/ destroy -auto-approve

Delete the state files and SSH key generated by the Terraform.

In [ ]:
rm -rf /tmp/ssh/.terraform /tmp/ssh/terraform.tfstate* /tmp/ssh/main.tf
rm -f /tmp/ssh/vault_init.txt

# Help and reference

- [Vault 1.0: How to auto-unseal and other new features](https://youtu.be/x9s0Uk9d510)
- [Seal Migration](https://www.vaultproject.io/docs/concepts/seal#seal-migration)
- [Vault Enterprise Auto Unseal](https://www.vaultproject.io/docs/concepts/seal)
- [Configuration: `awskms` Seal](https://www.vaultproject.io/docs/configuration/seal/awskms)
- [CLI - operator unseal](https://www.vaultproject.io/docs/commands/operator/unseal)
- Support KB - Seal Migration: https://support.hashicorp.com/hc/en-us/articles/360002040848-Seal-Migration
- Support KB - How-to rekey vault (recovery-keys) when using auto-unseal: https://support.hashicorp.com/hc/en-us/articles/4404364271763-How-to-rekey-vault-recovery-keys-when-using-auto-unseal